In [2]:
# imports 
import re
import sqlite3
import emoji

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from tqdm import tqdm
from matplotlib.ticker import MaxNLocator
from datetime import datetime 

tqdm.pandas()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [4]:
path_to_lds = '/Users/alexraudvee/Downloads/Twitter_Data.csv'

# path to the database 

path_to_db = '/Users/alexraudvee/Downloads/test.db'

In [4]:
df_for_acc = pd.read_csv(path_to_lds)
df_for_acc

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0
...,...,...
162975,why these 456 crores paid neerav modi not reco...,-1.0
162976,dear rss terrorist payal gawar what about modi...,-1.0
162977,did you cover her interaction forum where she ...,0.0
162978,there big project came into india modi dream p...,0.0


In [5]:
def converter(category):

    try:

        if int(category) == -1:
            return 'Negative'
        
        elif int(category) == 0:
            return 'Neutral'
        
        elif int(category) == 1:
            return 'Positive'
        
        else: 
            return None
        
    except:
        return None

In [ ]:
model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"

nlp = pipeline('sentiment-analysis', model = model_path)

In [ ]:
def get_sentiment(text):
    
    label = nlp(text)[0]['label']

    return label

In [6]:
df_for_acc['sentiment_given'] = df_for_acc['category'].progress_apply(lambda row: converter(row))
df_for_acc

100%|██████████| 162980/162980 [00:00<00:00, 764159.92it/s]


,clean_text,category,sentiment_given
0,when modi promised “minimum government maximum...,-1.0,Negative
1,talk all the nonsense and continue all the dra...,0.0,Neutral
2,what did just say vote for modi welcome bjp t...,1.0,Positive
3,asking his supporters prefix chowkidar their n...,1.0,Positive
4,answer who among these the most powerful world...,1.0,Positive
...,...,...,...
162975,why these 456 crores paid neerav modi not reco...,-1.0,Negative
162976,dear rss terrorist payal gawar what about modi...,-1.0,Negative
162977,did you cover her interaction forum where she ...,0.0,Neutral
162978,there big project came into india modi dream p...,0.0,Neutral


In [20]:
### take only tousand for the faster check and work
df_for_acc = df_for_acc.loc[:1000]

In [21]:
df_for_acc['sentiment_to_check'] = df_for_acc['clean_text'].progress_apply(lambda row: get_sentiment(row))

/var/folders/22/gxvfn3cx2_v64ngzhq52h2180000gn/T/ipykernel_25212/4128667393.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_acc['sentiment_to_check'] = 'Positive'


In [22]:
### EXTRACT TRUE AND CHECK VALUES

true_positive = int(df_for_acc[df_for_acc['sentiment_given'] == 'Positive']['sentiment_given'].count())
true_negative = int(df_for_acc[df_for_acc['sentiment_given'] == 'Negative']['sentiment_given'].count())
true_neutral = int(df_for_acc[df_for_acc['sentiment_given'] == 'Neutral']['sentiment_given'].count())

check_positive = int(df_for_acc[df_for_acc['sentiment_to_check'] == 'Positive']['sentiment_to_check'].count())
check_negative = int(df_for_acc[df_for_acc['sentiment_to_check'] == 'Negative']['sentiment_to_check'].count())
check_neutral = int(df_for_acc[df_for_acc['sentiment_to_check'] == 'Neutral']['sentiment_to_check'].count())


In [23]:

positives = true_positive - check_positive 
p = sqrt((positives/1000)**2)

negatives = true_negative - check_negative
neg = sqrt((negatives/1000)**2)

neutrals = true_neutral - check_neutral
neut = sqrt((neutrals/1000)**2)

(p, neg, neut)

(0.559, 0.225, 0.334)

## Another approach 

In [5]:
# connection to the database with sentiment scores

try:
    connection = sqlite3.connect(path_to_db)
    print('Succesfuly connected to the database')

    query = """
        SELECT name FROM sqlite_master  
        WHERE type='table';
        """

    cursor = connection.cursor()
    cursor.execute(query)
    list_of_tables = cursor.fetchall()

    print('---TABLE NAMES IN DB---')
    for table_name in list_of_tables:
        print('-----------------------')
        print(table_name[0])

    print('-----------------------')
    
except sqlite3.Error as e:
    print(e)


Succesfuly connected to the database
---TABLE NAMES IN DB---
-----------------------
users
-----------------------
tweets
-----------------------
tweets_geo
-----------------------
replies
-----------------------
retweets
-----------------------
quotes
-----------------------
hashtags
-----------------------
sqlite_sequence
-----------------------
symbols
-----------------------
user_mentions
-----------------------


In [6]:
# tweets table
def get_tweets(conn):
    query = """
        SELECT *
        FROM tweets
    """

    df = pd.read_sql(query, conn)
    return df


# create twees dataframe
df = get_tweets(conn=connection)

In [25]:
# create a random sample of tweets

df_sample_for_acc = df[(df['lang'] == 'en') & (df['tweet_type'] == 'original')].sample(200, replace=False)

In [26]:
df_sample_for_acc = df_sample_for_acc[['tweet_id', 'text']].reset_index()

In [53]:
df_sample_for_acc['label_int'] = None 

In [54]:
lable_int = [0, 0, 0, -1, 0, -1, -1, 0, 0, 0, -1, -1, 0, 0, ]

In [58]:
df_sample_for_acc.to_json('accuracy_check.json')